In [1]:
!pip install sentence_transformers openai
!pip install pinecone-client
# !pip3 install git+https://git@github.com/pinecone-io/pinecone-python-client.git

In [2]:
import os
os.environ['OPENAI_API_KEY'] = ""

In [3]:
#This is for embedding. In here, one LM model from huggingface used.

from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')


text ='Abc'
model.encode(text).tolist() #exmple how to do encoding.

[-0.00782928429543972,
 0.007352895103394985,
 -0.030943000689148903,
 0.041503604501485825,
 -0.07909709215164185,
 0.10331299155950546,
 0.04914174601435661,
 0.0030658103059977293,
 0.07064642757177353,
 -0.01297335047274828,
 -0.06321518868207932,
 -0.038936812430620193,
 -0.002514951629564166,
 -0.01407784316688776,
 -0.04177151992917061,
 -0.022063979879021645,
 -0.02587006986141205,
 -0.08123702555894852,
 -0.08500200510025024,
 -0.024818837642669678,
 -0.01968706212937832,
 0.020888442173600197,
 -0.03247418254613876,
 0.03827577084302902,
 -0.016411351040005684,
 0.03938812389969826,
 0.01233675703406334,
 0.03142799064517021,
 -0.05450307950377464,
 -0.12199369072914124,
 0.01660657860338688,
 0.06999657303094864,
 0.1426408737897873,
 -0.02005547098815441,
 -0.008527858182787895,
 -0.045705877244472504,
 -0.04244675114750862,
 -0.0309157595038414,
 0.05930527672171593,
 0.154644176363945,
 -0.02405787631869316,
 -0.00032021923107095063,
 0.06959789246320724,
 0.0536094419658

In [4]:
#Function to split long documents in to smaller parts
def split_text_into_chunks(plain_text, max_chars=2000):
    text_chunks = []
    current_chunk = ""
    for line in plain_text.split("\n"):
        if len(current_chunk) + len(line) + 1 <= max_chars:
            current_chunk += line + " "
        else:
            text_chunks.append(current_chunk.strip())
            current_chunk = line + " "
    if current_chunk:
        text_chunks.append(current_chunk.strip())
    return text_chunks

In [6]:
import pinecone
pinecone.init(api_key="", environment="") #Todo: Initialization of vector database module
index = pinecone.Index("") #Todo: Fill out with index name.

In [ ]:
def addData(corpusData):
    id  = index.describe_index_stats()['total_vector_count']
    for i in range(len(corpusData)):
        chunk=corpusData[i]
        chunkInfo=(str(id+i),
                model.encode(chunk).tolist(), #We are using the model to encode the original chunk of text.
                {'context': chunk}) #In metadata we are storing the original text here as context. 
        index.upsert(vectors=[chunkInfo])

In [ ]:
#This function is responsible for matching the input string with alread existing data on vector database.

def find_match(query,k):
    query_em = model.encode(query).tolist()
    result = index.query(query_em, top_k=k, includeMetadata=True)
    
    return [result['matches'][i]['metadata']['context'] for i in range(k)]

In [ ]:
def create_prompt(context,query):
  #Todo: Should be generated with the context/contexts we find by doing semantaic search
  pass

In [ ]:
def generate_answer(prompt):
  #Todo: Pass the generated prompt and pass it to gpt-3 to get answers.
  pass

In [ ]:
def user_query(query):
  #Todo: Make all the things together.
  pass


In [ ]:
user_query("How can I do this?")